In [2]:
import cirq
from cirq import LineQubit,Circuit, X, measure_each
from mitiq.observable.observable import Observable
from mitiq.observable.pauli import PauliString

# problem setup

In [5]:
qreg = [LineQubit(i) for i in range(2)]
circuit = Circuit(X.on_each(*qreg))
observable = Observable(PauliString("ZI"),PauliString("IZ"))

print(circuit)

0: ───X───

1: ───X───


# make executer

In [9]:
from functools import partial
import numpy as np
from cirq.experiments.single_qubit_readout_calibration_test import (
    NoisySingleQubitReadoutSampler,
)
from mitiq import MeasurementResult

def noisy_readout_executor(circuit,p0,p1,shots=8192)->MeasurementResult:
    simulator = NoisySingleQubitReadoutSampler(p0,p1)
    result = simulator.run(circuit,repetitions=shots)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings,qubit_indices=(0,1))

# evaluate expectation value

In [11]:
from mitiq.raw import execute as raw_execute

prob_flip = 0.25
noisy_executor = partial(noisy_readout_executor,p0=prob_flip,p1=prob_flip)
noisy_value = raw_execute(circuit,noisy_executor,observable)

ideal_executor = partial(noisy_readout_executor,p0=0,p1=0)
ideal_value = raw_execute(circuit,ideal_executor,observable)

error = abs((ideal_value-noisy_value) / ideal_value)

print(f"error without mitigation : {error:.3}")

error without mitigation : 0.505


# apply postselection

In [12]:
from mitiq.rem import post_select

#this method can only use if structure of problem has sysmetry

circuit_with_measurements = circuit.copy()
circuit_with_measurements.append(measure_each(*qreg))
noisy_measurements = noisy_executor(circuit_with_measurements)
print(f"Before postselection: {noisy_measurements.get_counts()}")

postselected_measurements = post_select(noisy_measurements,lambda bits: bits[0] == bits[1])
print(f"After postselection: {postselected_measurements.get_counts()}")

total_measurements = len(noisy_measurements.result)
discarded_measurements = total_measurements = len(postselected_measurements.result)
print(f"Discarded measurements: {discarded_measurements} ({discarded_measurements/total_measurements:.0%} of total)")

mitigated_result = observable._expectation_from_measurements([postselected_measurements])
error = abs((ideal_value-mitigated_result) / ideal_value)
print(f"Error with mitigation (PS): {error:.3}")

Before postselection: {'01': 1531, '11': 4573, '10': 1587, '00': 501}
After postselection: {'11': 4573, '00': 501}
Discarded measurements: 5074 (100% of total)
Error with mitigation (PS): 0.197


# apply REM

In [14]:
from mitiq.rem import generate_inverse_confusion_matrix
from mitiq import rem

inverse_confusion_matrix = generate_inverse_confusion_matrix(2,prob_flip,prob_flip)

mitigated_result = rem.execute_with_rem(circuit,noisy_executor,observable,inverse_confusion_matrix=inverse_confusion_matrix)

error = abs((ideal_value - mitigated_result)/ideal_value)
print(f"Error with mitigation (REM): {error:.3}")

Error with mitigation (REM): 0.0133
